In [ ]:
import os
import gc

from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np




# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "KEY"

# Clear existing DataFrames
if 'budget_data' in globals():
    del globals()['budget_data']
if 'employee_data' in globals():
    del globals()['employee_data']

# Force garbage collection
gc.collect()


# 1. Initialize LLM
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.3)


def generate_multi_year_budget():
    groups = [
        "IM IT", "IM IT-Employees", "IM IT-Contingent",
        "Tech COO-Employees", "Tech COO-Contingent",
        "Innovation-Employees", "Innovation-Contingent"
    ]
    
    data = {"Group": groups}
    
    # Simplified base values with rounded numbers
    base_values = {
        "Employees": np.array([200, 100, 50, 80, 40, 60, 30]) * 1_000_000,  # All in millions
        "Contingent": np.array([50, 20, 30, 15, 25, 10, 20]) * 1_000_000,    # All in millions
        "HC": np.array([8000, 4000, 2000, 3000, 1500, 2000, 1000])          # Headcounts
    }
    
    for year in [2025, 2026, 2027, 2028]:
        # Simplified yearly factors (5-10% growth/decline)
        yearly_factor = 1 + (np.random.rand(len(groups)) * 0.05 + 0.02)
        
        # Calculate current values (rounded to nearest 100,000) as integers
        current_budget = (base_values["Employees"] * yearly_factor)
        data[f"{year} Current $"] = (np.round(current_budget / 100_000) * 100_000).astype(int)
        
        current_hc = (base_values["HC"] * yearly_factor)
        data[f"{year} Current HC"] = (np.round(current_hc / 100) * 100).astype(int)
        
        # Simplified final factors (5-15% increase) as integers
        final_factor = 1.05 + (np.random.rand(len(groups)) * 0.10)
        data[f"{year} Final $"] = (np.round((data[f"{year} Current $"] * final_factor) / 100_000) * 100_000).astype(int)
        
        # HC final adjustment (±5%) as integers
        hc_factor = 0.95 + (np.random.rand(len(groups)) * 0.10)
        data[f"{year} Final HC"] = (np.round((data[f"{year} Current HC"] * hc_factor) / 100) * 100).astype(int)
        
        # Update bases for next year
        base_values["Employees"] = data[f"{year} Final $"]
        base_values["HC"] = data[f"{year} Final HC"]
    
    return pd.DataFrame(data)


def generate_budget_dataframe_detailed():
    # Define the groups
    groups = [        
        "IM IT- Employees",
        "IM IT- Contingent",
        "Tech COO- Employees",
        "Tech COO- Contingent",
        "Innovation- Employees",
        "Innovation- Contingent"
    ]
    
    # Create an empty DataFrame with the specified columns
    columns = ['Group']
    years = [2025, 2026, 2027, 2028]
    
    for year in years:
        columns.extend([
            f'{year} Current $',
            f'{year} Current HC',
            f'{year} Final $',
            f'{year} Final HC'
        ])
    
    df = pd.DataFrame(columns=columns)
    df['Group'] = groups
    
    # Fill the DataFrame with random data
    for year in years:
        # Current $ columns: random integers between 6,000,000 and 500,000,000
        current_dollars = np.random.randint(6_000_000, 500_000_000, size=len(groups))
        df[f'{year} Current $'] = current_dollars
        
        # Current HC columns: random integers between 2000 and 50,000
        df[f'{year} Current HC'] = np.random.randint(2000, 50_000, size=len(groups))
        
        # Final $ columns: equal or greater than Current $
        # We'll add a random percentage increase (0-20%) to ensure Final $ >= Current $
        increase_factor = 1 + np.random.rand(len(groups)) * 0.20  # 0-20% increase
        df[f'{year} Final $'] = (current_dollars * increase_factor).astype(int)
        
        # Final HC columns: same logic as Current HC (random between 2000 and 50,000)
        df[f'{year} Final HC'] = np.random.randint(2000, 50_000, size=len(groups))
        
    return df

# Generate data
budget_data = generate_multi_year_budget()

# Display the first few rows
budget_data.head()



,Group,2025 Current $,2025 Current HC,2025 Final $,2025 Final HC,2026 Current $,2026 Current HC,2026 Final $,2026 Final HC,2027 Current $,2027 Current HC,2027 Final $,2027 Final HC,2028 Current $,2028 Current HC,2028 Final $,2028 Final HC
0,IM IT,210700000,8400,227400000,8500,237000000,8900,259500000,9000,275700000,9600,294300000,9900,307400000,10300,323100000,9800
1,IM IT-Employees,105700000,4200,117900000,4400,124700000,4700,132200000,4800,141100000,5100,159800000,5100,164400000,5200,175100000,5300
2,IM IT-Contingent,52200000,2100,60000000,2100,61800000,2200,68300000,2300,70100000,2400,77700000,2500,81500000,2600,85700000,2700
3,Tech COO-Employees,81600000,3100,88100000,3200,91100000,3300,101100000,3300,104600000,3400,120200000,3500,124600000,3600,133500000,3700
4,Tech COO-Contingent,41800000,1600,46500000,1600,48600000,1700,53400000,1800,54600000,1800,62600000,1800,64200000,1800,73600000,1700


In [30]:
# 4. Create Agent with LLM
analyst = Agent(
    role="Senior Budget Analyst",
    goal="Analyze budget data and provide insights",
    backstory="Expert in financial and workforce planning with tech industry experience",
    verbose=True,
    allow_delegation=False,
    llm=llm  # Simple LLM integration here
)

# 5. Create Analysis Task
analysis_task = Task(
    description=f"""
    Analyze this budget data:
    {budget_data.to_markdown()}
    
    Provide:
    1. Overall trends 2025-2028
    2. Year-by-year key changes
    3. Group performance comparison
    4. Budget-to-HC ratio analysis
    5. 3 strategic recommendations
    """,
    agent=analyst,
    expected_output="Concise bullet-point report with clear insights and recommendations",
    output_file="budget_analysis_report.md"  # This saves to markdown file
)

# 6. Run Crew
crew = Crew(agents=[analyst], tasks=[analysis_task])
result = crew.kickoff()
print(result)

# Agent: Senior Budget Analyst
## Task: 
    Analyze this budget data:
    |    | Group                 |   2025 Current $ |   2025 Current HC |   2025 Final $ |   2025 Final HC |   2026 Current $ |   2026 Current HC |   2026 Final $ |   2026 Final HC |   2027 Current $ |   2027 Current HC |   2027 Final $ |   2027 Final HC |   2028 Current $ |   2028 Current HC |   2028 Final $ |   2028 Final HC |
|---:|:----------------------|-----------------:|------------------:|---------------:|----------------:|-----------------:|------------------:|---------------:|----------------:|-----------------:|------------------:|---------------:|----------------:|-----------------:|------------------:|---------------:|----------------:|
|  0 | IM IT                 |        210700000 |              8400 |      227400000 |            8500 |        237000000 |              8900 |      259500000 |            9000 |        275700000 |              9600 |      294300000 |            9900 |        307400000 |

Method 2 

In [ ]:
import pandas as pd
import numpy as np
from crewai import Agent, Task, Crew

# 1. Enhanced budget data generator with year-over-year trends
def generate_multi_year_budget():
    groups = [
        "IM IT", "IM IT-Employees", "IM IT-Contingent",
        "Tech COO-Employees", "Tech COO-Contingent",
        "Innovation-Employees", "Innovation-Contingent"
    ]
    
    data = {"Group": groups}
    base_values = {
        "Employees": np.random.randint(10_000_000, 300_000_000, len(groups)),
        "Contingent": np.random.randint(6_000_000, 100_000_000, len(groups)),
        "HC": np.random.randint(2000, 20_000, len(groups))
    }
    
    for year in [2025, 2026, 2027, 2028]:
        # Simulate annual changes (3-15% growth/decline)
        yearly_factor = 1 + (np.random.rand(len(groups)) * 0.12 - 0.03
        
        data[f"{year} Current $"] = (base_values["Employees"] * yearly_factor).astype(int)
        data[f"{year} Current HC"] = (base_values["HC"] * yearly_factor).astype(int)
        
        # Final $ is current $ plus random 5-25% increase
        final_factor = 1.05 + (np.random.rand(len(groups)) * 0.20)
        data[f"{year} Final $"] = (data[f"{year} Current $"] * final_factor).astype(int)
        
        # Final HC has ±10% variance from current
        hc_factor = 0.9 + (np.random.rand(len(groups)) * 0.2)
        data[f"{year} Final HC"] = (data[f"{year} Current HC"] * hc_factor).astype(int)
        
        # Update bases for next year with smaller variance
        base_values["Employees"] = data[f"{year} Final $"]
        base_values["HC"] = data[f"{year} Final HC"]
    
    return pd.DataFrame(data)

# 2. Create specialized agents
annual_analyst = Agent(
    role="Annual Budget Specialist",
    goal="Identify year-specific cost optimization opportunities",
    backstory="Expert in single-year financial planning with sharp eye for immediate savings",
    verbose=True
)

multi_year_analyst = Agent(
    role="Strategic Workforce Planner",
    goal="Find cross-year workforce optimization patterns",
    backstory="Seasoned strategist who identifies multi-year transformation opportunities",
    verbose=True
)

recommendation_engineer = Agent(
    role="Workflow Optimization Engineer",
    goal="Convert financial insights into actionable workforce plans",
    backstory="Industrial engineer specializing in tech workforce efficiency",
    verbose=True
)

# 3. Year-specific analysis task
def create_year_task(year, df):
    return Task(
        description=f"""Analyze {year} budget in isolation:
        - Current vs Final $ differences
        - HC allocation efficiency
        - Employee/Contingent cost ratios
        
        Data: {df[[col for col in df.columns if str(year) in col]].to_markdown()}""",
        agent=annual_analyst,
        expected_output=f"""3 key findings for {year}:
        1. [Cost inefficiency]
        2. [HC misallocation] 
        3. [Contingent ratio insight]""",
        output_file=f"{year}_analysis.md"
    )

# 4. Multi-year analysis task 
def create_multi_year_task(df):
    return Task(
        description=f"""Analyze 2025-2027 trends:
        - Cost trajectory
        - HC growth patterns
        - Contingent workforce % changes
        
        Full data: {df.to_markdown()}""",
        agent=multi_year_analyst,
        expected_output="""3 strategic insights:
        1. [Multi-year trend]
        2. [Compound savings opportunity]
        3. [Structural change recommendation]""",
        output_file="multi_year_analysis.md"
    )

# 5. Consolidated recommendation task
def create_recommendation_task(year_tasks, multi_year_task):
    return Task(
        description=f"""Synthesize all analyses:
        Annual Insights: {[t.output for t in year_tasks]}
        Strategic Insights: {multi_year_task.output}
        
        Create workforce plan covering:
        - Immediate 2025 actions
        - 2026-2027 transitions
        - 2028 preparedness""",
        agent=recommendation_engineer,
        expected_output="""Phased recommendation:
        ## 2025 Quick Wins
        - [Action 1]
        - [Action 2]
        
        ## 2026-2027 Transitions
        - [Change 1]
        - [Change 2]
        
        ## 2028 Readiness
        - [Preparation]""",
        output_file="final_recommendations.md",
        context=[*year_tasks, multi_year_task]
    )

# 6. Execute full analysis
def run_full_analysis():
    budget_df = generate_multi_year_budget()
    
    # Create tasks for each year
    year_tasks = [create_year_task(year, budget_df) for year in [2025, 2026, 2027]]
    
    # Multi-year analysis
    multi_year_task = create_multi_year_task(budget_df)
    
    # Final recommendations
    recommendation_task = create_recommendation_task(year_tasks, multi_year_task)
    
    crew = Crew(
        agents=[annual_analyst, multi_year_analyst, recommendation_engineer],
        tasks=[*year_tasks, multi_year_task, recommendation_task],
        verbose=2
    )
    
    results = crew.kickoff()
    
    # Generate comparison tables
    comparison = budget_df[[col for col in budget_df.columns if "$" in col]]
    print("\nBudget Comparison Table:\n", comparison.head())
    
    return results

# Run full analysis
final_output = run_full_analysis()
print("\nFinal Recommendations:\n", final_output)